In [12]:
import keras
from keras.applications import ResNet50
from keras.models import Sequential
import matplotlib.pyplot as plt
import os
from random import shuffle
from tqdm import tqdm
import tensorflow as tf
import cv2
import numpy as np 
from tensorflow.python.framework import graph_util
from keras.layers import *
from keras.optimizers import *
from sklearn.model_selection import train_test_split
import random
from keras.applications.resnet50 import preprocess_input
from keras.models import Sequential
from keras import optimizers
from keras.applications.resnet50 import ResNet50
from keras.models import Model
import keras

In [13]:
data1='D:\\val\\1coal'
data2='D:\\val\\2coal'
data3='D:\\val\\3coal'

In [16]:

def data_with_label_others():
    images=[]
    cnt = 0
    for i in tqdm(os.listdir(data1)):
        path=os.path.join(data1,i)
        img=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img=cv2.resize(img,(64,64))
        
        img1 = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
        img1=cv2.resize(img1,(224,224))
        images.append([np.array(img),np.array(img1),np.array([1,0,0])])
    cnt = 0
    for i in tqdm(os.listdir(data2)):
        path=os.path.join(data2,i)
        img=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img=cv2.resize(img,(64,64))
        
        img1 = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
        img1=cv2.resize(img1,(224,224))
        images.append([np.array(img),np.array(img1),np.array([0,1,0])])
    cnt = 0
    for i in tqdm(os.listdir(data3)):
        path=os.path.join(data3,i)
        img=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img=cv2.resize(img,(64,64))
        
        img1 = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
        img1=cv2.resize(img1,(224,224))
        images.append([np.array(img),np.array(img1),np.array([0,0,1])])
        
    return images

data = data_with_label_others()

random.shuffle(data)

xo = np.array([i[0] for i in data]).reshape(-1,64,64,1)
xo = xo/255
xr = np.array([i[1] for i in data]).reshape(-1,224,224,3)
xr = preprocess_input(xr)
yo = np.array([i[2] for i in data])




100%|██████████████████████████████████████████████████████████████████████████████████| 70/70 [00:04<00:00, 15.88it/s]

 26%|█████████████████████▋                                                            | 24/91 [00:01<00:05, 12.76it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 91/91 [00:06<00:00, 14.94it/s]

In [20]:
# Loads the weights
def create_model_cnn():
  model=Sequential()
  model.add(InputLayer(input_shape=[64,64,1]))

  model.add(Conv2D(filters=32,kernel_size=5,strides=1,padding='same',activation='relu'))
  model.add(MaxPool2D(pool_size=5,padding='same'))
          
  model.add(Conv2D(filters=32,kernel_size=5,strides=2,padding='same',activation='relu'))
  model.add(MaxPool2D(pool_size=5,padding='same'))
          
  model.add(Conv2D(filters=64,kernel_size=3,strides=2,padding='same',activation='relu'))
  model.add(MaxPool2D(pool_size=8,padding='same'))

  model.add(Conv2D(filters=128,kernel_size=3,strides=1,padding='same',activation='relu'))
  model.add(MaxPool2D(pool_size=5,padding='same'))
          

  model.add(Flatten())


  model.add(Dense(2048,activation='relu'))
  model.add(Dropout(0.50))
  model.add(Dense(1024,activation='relu'))
  model.add(Dropout(0.50))
  model.add(Dense(512,activation='relu'))
  model.add(Dropout(0.50))
  model.add(Dense(3,activation='softmax'))
  optimizer=Adam(lr=1e-3)
          
  history1=model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])
  return model

checkpoint_path = "cp.ckpt"
model_cnn=create_model_cnn()

model_cnn.load_weights(checkpoint_path)

# Re-evaluate the model
loss,acc = model_cnn.evaluate(xo, yo, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

Restored model, accuracy: 94.06%


In [21]:
def create_model_resnet():
    
    restnet = ResNet50(include_top=False, weights='imagenet', input_shape=(224,224,3))
    output = restnet.layers[-1].output
    output = keras.layers.Flatten()(output)
    restnet = Model(restnet.input, output=output)
    for layer in restnet.layers:
        layer.trainable = False
    import pandas as pd
    restnet.trainable = True
    set_trainable = False
    for layer in restnet.layers:
        if layer.name in ['res5c_branch2b', 'res5c_branch2c', 'activation_97']:
            set_trainable = True
        if set_trainable:
            layer.trainable = True
        else:
            layer.trainable = False
    layers = [(layer, layer.name, layer.trainable) for layer in restnet.layers]
  
    model_finetuned = Sequential()
    model_finetuned.add(restnet)
    model_finetuned.add(Dense(512, activation='relu', input_dim=(224,224,1)))
    model_finetuned.add(Dropout(0.5))
    model_finetuned.add(Dense(3, activation='sigmoid'))
    history2=model_finetuned.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.RMSprop(lr=1e-5),
                  metrics=['accuracy'])
    return model_finetuned




checkpoint_path = "cpresnet.ckpt"
model_resnet=create_model_resnet()

model_resnet.load_weights(checkpoint_path)

# Re-evaluate the model
loss,acc = model_resnet.evaluate(xr,  yo, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

C:\Users\Abhilasha\Anaconda2\envs\tensorflow_env\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Model` call to the Keras 2 API: `Model(Tensor("in..., outputs=Tensor("fl...)`
  


Restored model, accuracy: 78.08%


In [37]:
#taking two images to check their similarity
def create_model_siamese():

    left_input = Input((64,64,1))
    right_input = Input((64,64,1))

# Base network architecture

    convnet=Sequential()
    convnet.add(InputLayer(input_shape=[64,64,1]))
    convnet.add(Conv2D(filters=32,kernel_size=5,strides=1,padding='same',activation='relu'))
    convnet.add(MaxPool2D(pool_size=5,padding='same'))   
    convnet.add(Conv2D(filters=32,kernel_size=5,strides=1,padding='same',activation='relu'))
    convnet.add(MaxPool2D(pool_size=5,padding='same'))  
    convnet.add(Conv2D(filters=64,kernel_size=3,strides=1,padding='same',activation='relu'))
    convnet.add(MaxPool2D(pool_size=8,padding='same'))
    convnet.add(Conv2D(filters=128,kernel_size=3,strides=1,padding='same',activation='relu'))
    convnet.add(MaxPool2D(pool_size=5,padding='same'))   
    convnet.add(Conv2D(filters=256,kernel_size=3,strides=1,padding='same',activation='relu'))
    convnet.add(Flatten())

    convnet.add(Dense(1024,activation='relu'))
    convnet.add(Dropout(0.20))
    convnet.add(Dense(512,activation='relu'))
    convnet.add(Dropout(0.25))
    convnet.add(Dense(64,activation='sigmoid'))

#generating the encodings for two input images
    encoded_l = convnet(left_input)
    encoded_r = convnet(right_input)

# Getting the L1 Distance between the 2 encodings
    L1_layer = Lambda(lambda tensor:K.abs(tensor[0] - tensor[1]))

# Add the distance function to the network
    L1_distance = L1_layer([encoded_l, encoded_r])

    prediction = Dense(1,activation='sigmoid')(L1_distance)
    siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)

    optimizer = Adam(0.0001, decay=2.5e-4)
    history3=siamese_net.compile(loss="binary_crossentropy",optimizer=optimizer,metrics=['accuracy'])
    return siamese_net





test_left = []
test_right = []
test_targets = []
img1 = xo[68]
for i in range(yo.shape[0]):
    test_left.append(img1)
    test_right.append(xo[i])
    #print(test_left[i].shape)
    #print(test_right[i].shape)
    if np.argmax(yo[68])==np.argmax(yo[i]):
        test_targets.append(1.)
    else:
        test_targets.append(0.)


test_left = np.squeeze(np.array(test_left))
test_right = np.squeeze(np.array(test_right))
test_targets = np.squeeze(np.array(test_targets))

test_left = np.array([i for i in test_left]).reshape(-1,64,64,1)
test_right = np.array([i for i in test_right]).reshape(-1,64,64,1)
print(test_left.shape)

model_siamese = create_model_siamese()
# Loads the weights
checkpoint_path='cpss2.ckpt'
model_siamese.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model_siamese.evaluate([test_left,test_right], test_targets, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

(219, 64, 64, 1)
Restored model, accuracy: 89.50%


In [62]:
def ensemble():
    p=0
    a1= cv2.imread("D://fintest//1coal//pic1crop.jpg", cv2.IMREAD_GRAYSCALE)
    a1= cv2.resize(a1,(64,64))
    a1=np.array(a1).reshape(-1,64,64,1)
    a1= a1/255
    
    a2= cv2.imread("D://fintest//2coal//pic1crop.jpg", cv2.IMREAD_GRAYSCALE)
    a2= cv2.resize(a2,(64,64))
    a2=np.array(a2).reshape(-1,64,64,1)
    a2= a2/255
    
    a3= cv2.imread("D://fintest//3coal//pic1crop.jpg", cv2.IMREAD_GRAYSCALE)
    a3= cv2.resize(a3,(64,64))
    a3=np.array(a3).reshape(-1,64,64,1)
    a3= a3/255
  
    for i in range(yo.shape[0]):
        o=[]
        img=np.array(xo[i]).reshape(-1,64,64,1)
        o.append(np.argmax(model_cnn.predict(img)))
        img=np.array(xr[i]).reshape(-1,224,224,3)
        o.append(np.argmax(model_resnet.predict(img)))
        
        img=np.array(xo[i]).reshape(-1,64,64,1)
        b1= model_siamese.predict([a1,img])
        b2= model_siamese.predict([a2,img])
        b3= model_siamese.predict([a3,img])
        
        if(b1>=b2 and b1>=b3):
            o.append(0)
        elif (b2>=b1 and b2>=b3):
            o.append(1)
        else:
            o.append(2)
        ans=max(set(o), key=o.count)
        print(i,ans)
        if(ans==np.argmax(yo[i])):
            p+=1
            
    print(p/yo.shape[0]*100)
        
        
        

In [63]:
ensemble()

0 1
1 1
2 2
3 2
4 0
5 1
6 2
7 2
8 2
9 1
10 0
11 2
12 2
13 0
14 0
15 2
16 2
17 1
18 0
19 1
20 0
21 1
22 0
23 2
24 0
25 0
26 0
27 2
28 0
29 1
30 0
31 2
32 1
33 2
34 2
35 0
36 2
37 0
38 1
39 2
40 0
41 1
42 1
43 0
44 2
45 0
46 2
47 0
48 0
49 2
50 2
51 1
52 2
53 0
54 2
55 1
56 1
57 0
58 2
59 0
60 0
61 1
62 2
63 0
64 1
65 1
66 2
67 0
68 1
69 2
70 0
71 0
72 2
73 0
74 2
75 2
76 1
77 2
78 2
79 0
80 2
81 0
82 2
83 0
84 2
85 0
86 2
87 2
88 2
89 2
90 0
91 2
92 0
93 2
94 2
95 2
96 0
97 2
98 0
99 2
100 0
101 0
102 2
103 2
104 0
105 1
106 2
107 0
108 0
109 2
110 1
111 2
112 2
113 2
114 2
115 0
116 2
117 1
118 2
119 0
120 0
121 2
122 2
123 0
124 0
125 0
126 0
127 1
128 1
129 1
130 0
131 1
132 2
133 2
134 2
135 1
136 1
137 0
138 1
139 2
140 2
141 1
142 2
143 1
144 2
145 0
146 0
147 2
148 2
149 2
150 0
151 0
152 2
153 2
154 2
155 1
156 0
157 1
158 0
159 0
160 2
161 0
162 2
163 0
164 1
165 1
166 2
167 2
168 0
169 0
170 1
171 1
172 2
173 0
174 1
175 0
176 0
177 1
178 2
179 0
180 0
181 1
182 1
183 2
184 2
